# 원, 삼각형, 사각형 구분하기
## 3. Data Augmentation을 사용한 CNN 모델

In [1]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from keras.models import Sequential
from keras.layers import Dense, Flatten, Conv2D, MaxPooling2D
from keras.preprocessing.image import ImageDataGenerator

Using TensorFlow backend.


In [2]:
# seed 값 설정
seed = 2020
np.random.seed(seed)
tf.random.set_seed(seed)

### 데이터 부풀리기(Augmentation)

In [3]:
from keras.preprocessing.image import array_to_img, img_to_array, load_img

In [6]:
train_datagen = ImageDataGenerator(rescale=1./255, 
                                   rotation_range=15,
                                   width_shift_range=0.1,
                                   height_shift_range=0.1,
                                   shear_range=0.5,
                                   zoom_range=[0.8, 2.0],
                                   horizontal_flip=True,
                                   vertical_flip=True,
                                   fill_mode='nearest')

In [7]:
data_aug_gen = ImageDataGenerator(rescale=1./255, 
                                  rotation_range=15,
                                  width_shift_range=0.1,
                                  height_shift_range=0.1,
                                  shear_range=0.5,
                                  zoom_range=[0.8, 2.0],
                                  horizontal_flip=True,
                                  vertical_flip=True,
                                  fill_mode='nearest')

In [8]:
img = load_img('hard_handwriting_shape/train/triangle/triangle001.png')
x = img_to_array(img)
x = x.reshape((1,) + x.shape)

i = 0

# 무한 for loop이니 주의!!!
for batch in train_datagen.flow(x, batch_size=1, save_to_dir='preview', 
                                save_prefix='tri', save_format='png'):
    i += 1
    if i > 30: 
        break

### 데이터셋 생성하기

In [9]:
train_generator = train_datagen.flow_from_directory(
        'hard_handwriting_shape/train',
        target_size=(24, 24),
        batch_size=3,
        class_mode='categorical')

test_datagen = ImageDataGenerator(rescale=1./255)

test_generator = test_datagen.flow_from_directory(
        'hard_handwriting_shape/test',
        target_size=(24, 24),    
        batch_size=3,
        class_mode='categorical')

Found 45 images belonging to 3 classes.
Found 15 images belonging to 3 classes.


### 모델 구성하기

In [10]:
model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 3),
                 activation='relu', input_shape=(24,24,3)))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dense(3, activation='softmax'))
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 22, 22, 32)        896       
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 20, 20, 64)        18496     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 10, 10, 64)        0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 6400)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 128)               819328    
_________________________________________________________________
dense_2 (Dense)              (None, 3)                 387       
Total params: 839,107
Trainable params: 839,107
Non-trainable params: 0
________________________________________________

### 모델 학습과정 설정하기

In [11]:
model.compile(loss='categorical_crossentropy', 
              optimizer='adam', metrics=['accuracy'])

### 모델 학습시키기
- 첫번째 인자 : 훈련데이터셋을 제공할 제네레이터를 지정. 본 예제에서는 앞서 생성한 train_generator으로 지정.
- steps_per_epoch : 한 epoch에 사용한 스텝 수를 지정. 총 45개의 훈련 샘플이 있고 배치사이즈가 3이므로 15 스텝으로 지정.
- epochs : 전체 훈련 데이터셋에 대해 학습 반복 횟수를 지정. 200번을 반복적으로 학습.
- validation_data : 검증데이터셋을 제공할 제네레이터를 지정. 본 예제에서는 앞서 생성한 validation_generator으로 지정.
- validation_steps : 한 epoch 종료 시 마다 검증할 때 사용되는 검증 스텝 수를 지정. 홍 15개의 검증 샘플이 있고 배치사이즈가 3이므로 5 스텝으로 지정.

In [12]:
model.fit_generator(
        train_generator,
        steps_per_epoch=15,
        epochs=200,
        validation_data=test_generator,
        validation_steps=5)

Epoch 1/200
15/15 [==============================] - 2s 163ms/step - loss: 1.4515 - accuracy: 0.2000 - val_loss: 1.1237 - val_accuracy: 0.3333
Epoch 2/200
15/15 [==============================] - 0s 19ms/step - loss: 1.0932 - accuracy: 0.3333 - val_loss: 1.0001 - val_accuracy: 0.4667
Epoch 3/200
15/15 [==============================] - 0s 20ms/step - loss: 1.0567 - accuracy: 0.3556 - val_loss: 1.1277 - val_accuracy: 0.4667
Epoch 4/200
15/15 [==============================] - 0s 20ms/step - loss: 0.9164 - accuracy: 0.5556 - val_loss: 0.9897 - val_accuracy: 0.6000
Epoch 5/200
15/15 [==============================] - 0s 20ms/step - loss: 0.7665 - accuracy: 0.5556 - val_loss: 0.6323 - val_accuracy: 0.7333
Epoch 6/200
15/15 [==============================] - 0s 20ms/step - loss: 0.8457 - accuracy: 0.6000 - val_loss: 0.6109 - val_accuracy: 0.4667
Epoch 7/200
15/15 [==============================] - 0s 21ms/step - loss: 0.7514 - accuracy: 0.5778 - val_loss: 0.6534 - val_accuracy: 0.4000
Epoch

15/15 [==============================] - 0s 20ms/step - loss: 0.1579 - accuracy: 0.9556 - val_loss: 1.1828 - val_accuracy: 0.5333
Epoch 59/200
15/15 [==============================] - 0s 20ms/step - loss: 0.1197 - accuracy: 0.9556 - val_loss: 0.6282 - val_accuracy: 0.6000
Epoch 60/200
15/15 [==============================] - 0s 20ms/step - loss: 0.0144 - accuracy: 1.0000 - val_loss: 0.1229 - val_accuracy: 0.8000
Epoch 61/200
15/15 [==============================] - 0s 20ms/step - loss: 0.0650 - accuracy: 0.9556 - val_loss: 0.1086 - val_accuracy: 0.8000
Epoch 62/200
15/15 [==============================] - 0s 20ms/step - loss: 0.0137 - accuracy: 1.0000 - val_loss: 4.3662 - val_accuracy: 0.8000
Epoch 63/200
15/15 [==============================] - 0s 20ms/step - loss: 0.1020 - accuracy: 0.9556 - val_loss: 4.4460 - val_accuracy: 0.8000
Epoch 64/200
15/15 [==============================] - 0s 20ms/step - loss: 0.0211 - accuracy: 1.0000 - val_loss: 4.9519 - val_accuracy: 0.7333
Epoch 65/200

15/15 [==============================] - 0s 19ms/step - loss: 5.5613e-04 - accuracy: 1.0000 - val_loss: 0.0034 - val_accuracy: 0.6667
Epoch 116/200
15/15 [==============================] - 0s 19ms/step - loss: 0.0034 - accuracy: 1.0000 - val_loss: 2.6130 - val_accuracy: 0.6667
Epoch 117/200
15/15 [==============================] - 0s 19ms/step - loss: 0.1890 - accuracy: 0.9556 - val_loss: 2.4456 - val_accuracy: 0.6667
Epoch 118/200
15/15 [==============================] - 0s 19ms/step - loss: 0.1128 - accuracy: 0.9556 - val_loss: 4.4612 - val_accuracy: 0.6667
Epoch 119/200
15/15 [==============================] - 0s 19ms/step - loss: 0.0248 - accuracy: 1.0000 - val_loss: 1.6120 - val_accuracy: 0.6667
Epoch 120/200
15/15 [==============================] - 0s 19ms/step - loss: 0.0474 - accuracy: 0.9778 - val_loss: 0.0022 - val_accuracy: 0.6667
Epoch 121/200
15/15 [==============================] - 0s 19ms/step - loss: 0.0719 - accuracy: 0.9556 - val_loss: 1.6013e-05 - val_accuracy: 0.666

Epoch 172/200
15/15 [==============================] - 0s 20ms/step - loss: 0.0082 - accuracy: 1.0000 - val_loss: 3.7413 - val_accuracy: 0.7333
Epoch 173/200
15/15 [==============================] - 0s 19ms/step - loss: 0.0011 - accuracy: 1.0000 - val_loss: 7.2629 - val_accuracy: 0.6667
Epoch 174/200
15/15 [==============================] - 0s 20ms/step - loss: 4.2837e-04 - accuracy: 1.0000 - val_loss: 3.8874 - val_accuracy: 0.6667
Epoch 175/200
15/15 [==============================] - 0s 20ms/step - loss: 0.0050 - accuracy: 1.0000 - val_loss: 0.0030 - val_accuracy: 0.6667
Epoch 176/200
15/15 [==============================] - 0s 20ms/step - loss: 8.5255e-04 - accuracy: 1.0000 - val_loss: 2.1378e-05 - val_accuracy: 0.6667
Epoch 177/200
15/15 [==============================] - 0s 21ms/step - loss: 0.0039 - accuracy: 1.0000 - val_loss: 5.6603 - val_accuracy: 0.6667
Epoch 178/200
15/15 [==============================] - 0s 20ms/step - loss: 0.0032 - accuracy: 1.0000 - val_loss: 1.9950 - v

### 모델 평가하기

In [13]:
print("-- Evaluate --")
scores = model.evaluate_generator(test_generator, steps=5)
print("%s: %.2f%%" %(model.metrics_names[1], scores[1]*100))

-- Evaluate --
accuracy: 53.33%


### 모델 사용하기

In [14]:
print("-- Predict --")
output = model.predict_generator(test_generator, steps=5)
np.set_printoptions(formatter={'float': lambda x: "{0:0.3f}".format(x)})
print(test_generator.class_indices)
print(output)

-- Predict --
{'circle': 0, 'rectangle': 1, 'triangle': 2}
[[0.000 0.000 1.000]
 [0.000 0.000 1.000]
 [0.001 0.451 0.548]
 [0.000 0.000 1.000]
 [0.000 0.000 1.000]
 [0.000 0.000 1.000]
 [0.000 1.000 0.000]
 [0.024 0.668 0.309]
 [1.000 0.000 0.000]
 [1.000 0.000 0.000]
 [0.240 0.000 0.760]
 [0.000 0.000 1.000]
 [0.002 0.320 0.678]
 [0.000 0.002 0.998]
 [0.001 0.104 0.895]]
